In [12]:
# import pytorch tools
import torch
import torch.nn as nn
import torch.optim as optim

# other ones also ...
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [2]:
# loading the data

diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target.reshape(-1, 1)

In [3]:
# Standardize the features

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [4]:
# Split the dataset into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

In [5]:
# Convert data to PyTorch tensors

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [6]:
# defining the nn model

class DiabetesNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(DiabetesNN, self).__init__()
    self.fc1 = nn.Linear(
        in_features = input_size,
        out_features = hidden_size,
    )
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(
        in_features = hidden_size,
        out_features = output_size,
    )

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [7]:
# Instantiate the model

input_size = X.shape[1]  # Number of features in the diabetes dataset
hidden_size = 8
output_size = 1  # Regression output
model = DiabetesNN(input_size, hidden_size, output_size)

In [8]:
# Loss function and optimizer

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
# training the model

num_epochs = 1000
for epoch in range(num_epochs):
  # forward pass
  outputs = model(X_train)
  loss = criterion(outputs, y_train)

  # backprop
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if (epoch+1) % 100 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [100/1000], Loss: 29386.2891
Epoch [200/1000], Loss: 28941.9258
Epoch [300/1000], Loss: 28193.3398
Epoch [400/1000], Loss: 27171.5430
Epoch [500/1000], Loss: 25923.2949
Epoch [600/1000], Loss: 24428.3340
Epoch [700/1000], Loss: 22676.7676
Epoch [800/1000], Loss: 20773.6934
Epoch [900/1000], Loss: 18834.7930
Epoch [1000/1000], Loss: 16938.0508


In [13]:
# Evaluate the model on the test set

with torch.no_grad():
    predicted = model(X_test)
    mse = mean_squared_error(y_test.numpy(), predicted.numpy())
    r2 = r2_score(y_test.numpy(), predicted.numpy())

    print(f'Mean Squared Error on Test Set: {mse:.4f}')
    print(f'R2 score on Test Set: {r2:.4f}')

Mean Squared Error on Test Set: 15036.6699
R2 score on Test Set: -1.8381
